![imagenes](logo.png)

# Gráficos de puntos

Los gráficos más simples de los 5NG son los diagramas de puntos, también llamados diagramas bivariados o de dispersión. Te permiten visualizar la relación entre dos variables numéricas. Si bien es posible que ya estés familiarizado con los diagramas de puntos, veámoslos a través de la lente de la gramática de gráficos. 

---
---
**Ejemplo 2.2.1**

Visualizaremos la relación entre las siguientes dos variables numéricas del dataframe [alaska.csv](https://github.com/scidatmath2020/CDD_R_2022/blob/main/data/alaska.csv): 

* ``dep_delay``: retraso de salida en el eje horizontal "x" y
* ``arr_delay:`` retraso de llegada en el eje vertical "y"

para vuelos de Alaska Airlines que salieron de Nueva York en 2013. En otras palabras, ``alaska_flights`` no consta de todos los vuelos que salieron de Nueva York en 2013, sino solo de aquellos vuelos en los que la aerolínea (``carrier``) es AS (que es el código de aerolínea de Alaska Airlines).

In [ ]:
library(tidyverse)

In [ ]:
alaska_flights <- read.csv("alaska.csv")

In [ ]:
str(alaska_flights)

---
---

## Gráfico de puntos con ggplot

Ahora repasemos el código que creará el diagrama de dispersión deseado, teniendo en cuenta la gramática del marco de gráficos. Echemos un vistazo al código y desglosémoslo pieza por pieza.

In [ ]:
ggplot(data = alaska_flights, mapping = aes(x = dep_delay, y = arr_delay)) + 
  geom_point()

Dentro de la función ``ggplot()``, especificamos dos de los componentes de la gramática de gráficos como argumentos (es decir, entradas):

* Los datos son el dataframe ``alaska_flights``. Por lo tanto ``data = alaska_flights``.
* El mapeo estético configurando ``mapping = aes(x = dep_delay, y = arr_delay)``. Específicamente, la variable ``dep_delay`` se asigna a la posición x de la estética, mientras que la variable ``arr_delay`` se asigna a la posición y.

Luego agregamos una capa a la llamada de función ``ggplot()`` usando el signo ``+``. La capa añadida en cuestión especifica el tercer componente de la gramática: el objeto geométrico. En este caso, el objeto geométrico se establece en puntos especificando ``geom_point()``. Después de ejecutar estas dos líneas de código en su consola, notará dos resultados: un mensaje de advertencia y el gráfico anterior.

Primero analicemos el gráfico. Observe que existe una relación positiva entre ``dep_delay`` y ``arr_delay``: a medida que aumentan los retrasos en las salidas, los retrasos en las llegadas tienden a aumentar también. Observe también la gran masa de puntos agrupados cerca de (0, 0), el punto que indica vuelos que no partieron ni llegaron tarde.

Dirijamos nuestra atención al mensaje de advertencia. **R** nos alerta sobre el hecho de que se ignoraron cinco filas. Para estas 5 filas, faltaba el valor de ``dep_delay`` o ``arr_delay`` o ambos (registrado en **R** como ``NA``) y, por lo tanto, estas filas se ignoraron en nuestro diagrama.

In [ ]:
alaska_flights[is.na(alaska_flights$dep_delay) | is.na(alaska_flights$arr_delay),]

Antes de continuar, hagamos algunas observaciones más sobre este código que creó el diagrama de dispersión. Tengamos en cuenta que el signo ``+`` aparece al final de las líneas y no al principio. Obtendrás un error en **R** si lo pones al principio de una línea. 

Para enfatizar la importancia de agregar la capa que especifica el objeto geométrico, considera el siguiente código donde no se agregan capas. ¡Debido a que no se especificó el objeto geométrico, tenemos un gráfico en blanco que no es muy útil!

In [ ]:
ggplot(data = alaska_flights, mapping = aes(x = dep_delay, y = arr_delay))

---
---
**Ejercicios.**

**2.2.1.** ¿Cuáles son algunas razones prácticas por las que ``dep_delay`` y ``arr_delay`` tienen una relación positiva?

**2.2.2** ¿Por qué crees que hay un grupo de puntos cerca de (0, 0)? ¿A qué corresponde (0, 0) en términos de los vuelos de Alaska Air?

**2.2.3** ¿Qué otras características del gráfico te llaman la atención?

**2.2.4** Crea un nuevo diagrama de dispersión usando diferentes variables del dataframe ``alaska_flights`` modificando el ejemplo dado.

## Superposición

La gran masa de puntos cerca de (0, 0) en el gráfico puede causar cierta confusión, ya que es difícil saber el número real de puntos que se trazan. Este es el resultado de un fenómeno llamado *superposición*. Como se puede suponer, esto corresponde a puntos que se trazan uno encima del otro una y otra vez. Cuando se produce un trazado excesivo, es difícil saber el número de puntos que se están trazando. Hay dos métodos para abordar el problema de la superposición de gráficos. Ya sea por

1. Ajuste de la transparencia de los puntos o
2. Agregar un poco de "nerviosismo" aleatorio o "empujones" aleatorios a cada uno de los puntos.

### Método 1: cambiar la transparencia

La primera forma de abordar la superposición es cambiar la transparencia de los puntos configurando el argumento ``alfa`` en ``geom_point()``. Podemos cambiar el argumento alfa para que sea cualquier valor entre 0 y 1, donde 0 establece que los puntos sean 100% transparentes y 1 establece que los puntos sean 100% opacos. De forma predeterminada, ``alfa`` se establece en 1. En otras palabras, si no establecemos explícitamente un valor ``alfa``, **R** usará ``alfa = 1``.

Observe cómo el siguiente código es idéntico al código de la anterior que creó el diagrama de dispersión con superposición, pero con ``alpha = 0.2`` agregado a la función ``geom_point()``:

In [ ]:
ggplot(data = alaska_flights, mapping = aes(x = dep_delay, y = arr_delay)) + 
  geom_point(alpha = 0.2)

La característica clave a tener en cuenta en este gráfico es que la transparencia de los puntos es acumulativa: las áreas con un alto grado de superposición son más oscuras, mientras que las áreas con un grado más bajo son menos oscuras. Tenga en cuenta además que no hay ``aes()`` alrededor de ``alpha = 0.2``. Esto se debe a que no estamos asignando una variable a un atributo estético, sino simplemente cambiando la configuración predeterminada de ``alfa``. De hecho, recibirá un error si intenta cambiar la segunda línea y hacer ``geom_point(aes(alpha = 0.2))``.

### Método 2: agitar los puntos

La segunda forma de abordar la superposición es alterando todos los puntos. Esto significa dar a cada punto un pequeño "empujón" en una dirección aleatoria. Puede pensar en "temblor" como sacudir un poco los puntos de la trama. Ilustremos usando un ejemplo simple primero. Digamos que tenemos un dataframe con 4 filas idénticas de valores x e y: (0,0), (0,0), (0,0) y (0,0). En la siguiente figura, presentamos tanto el diagrama de dispersión regular de estos 4 puntos (a la izquierda) como su contraparte fluctuante (a la derecha).

![imagenes](im001.png)

En el diagrama de dispersión regular de la izquierda, observa que los 4 puntos están superpuestos uno encima del otro. Si bien sabemos que se están trazando 4 valores, este hecho podría no ser evidente para otros. En el diagrama de dispersión con fluctuaciones de la derecha, ahora es claramente evidente que este diagrama involucra cuatro puntos, ya que a cada punto se le da un “empujón” aleatorio.

Ten en cuenta, sin embargo, que el "empujón" es estrictamente una herramienta de visualización; incluso después de crear un diagrama de dispersión fluctuante, los valores originales guardados en el dataframe original permanecen sin cambios.

Para crear un diagrama de dispersión con fluctuaciones, en lugar de usar ``geom_point()``, usamos ``geom_jitter()``. Observe cómo el siguiente código es muy similar al código que creó el diagrama de dispersión con sobreposición original, pero con ``geom_point()`` reemplazado por ``geom_jitter()``.

In [ ]:
ggplot(data = alaska_flights, mapping = aes(x = dep_delay, y = arr_delay)) + 
  geom_jitter(width = 30, height = 30)

Para especificar cuánto ruido agregar, ajustamos los argumentos de ancho ``width`` y alto ``height``a ``geom_jitter()``. Esto corresponde a la fuerza con la que le gustaría sacudir la gráfica en unidades del eje x horizontal y unidades del eje y vertical, respectivamente. En este caso, ambos ejes están en minutos. ¿Cuánto ruido debemos agregar usando los argumentos de ancho y alto? Por un lado, es importante agregar suficiente fluctuación para romper cualquier superposición en los puntos, pero por otro lado, no tanto como para alterar completamente el patrón original en los puntos.

Como se puede ver en la gráfica resultante, en este caso el ruido realmente no proporciona mucha información nueva. En este caso particular, se puede argumentar que cambiar la transparencia de los puntos configurando ``alfa`` demostró ser más efectivo. ¿Cuándo sería mejor usar un diagrama de dispersión con fluctuaciones? ¿Cuándo sería mejor alterar la transparencia de los puntos? No existe una única respuesta correcta que se aplique a todas las situaciones. Necesitas hacer una elección subjetiva y ser dueño de esa elección. Sin embargo, al menos cuando se enfrenta a una trama excesiva, le sugerimos que haga ambos tipos de tramas y vea cuál enfatiza mejor el punto que está tratando de hacer.

## En resumen...

Los gráficos de puntos sirven para visualizar las relaciones entre dos variables numéricas. Son uno de los modos de graficación más usuales que existen debido a que proporcionan un medio visual para observar la tendencia de una variable numérica respecto de otra. Sin embargo, si intentas crear un gráfico de puntos donde alguna de las variables no es numérica, puedes obtener resultados extraños (aunque no necesariamente erróneos).

Con conjuntos de datos medianos o grandes, es posible que debas experimentar con las diferentes modificaciones a los diagramas de dispersión que vimos, como cambiar la transparencia/opacidad de los puntos o agitar los puntos. Este ajuste suele ser una parte divertida de la visualización de datos, ya que tendrás la oportunidad de ver cómo surgen diferentes relaciones a medida que juegas con tus gráficos.